In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('brain_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_data')

In [24]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

print("All libraries imported successfully!")

All libraries imported successfully!


In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

train_dir = "/content/brain_dataset/Training"
test_dir  = "/content/brain_dataset/Testing"

IMG_SIZE = 224
BATCH_SIZE = 16   # medical datasets are usually smaller


In [27]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    shuffle=False
)


Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [28]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    x = tf.image.grayscale_to_rgb(x)  # DenseNet expects 3 channels
    return x, y

train_ds = train_ds.map(preprocess)
test_ds  = test_ds.map(preprocess)

# performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)


In [29]:
base_model = tf.keras.applications.DenseNet121(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False  # Phase 1: freeze


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [30]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(4, activation="softmax") # Changed to 4 classes with softmax for multi-class classification
])

In [31]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy", # Changed loss function for multi-class integer labels
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

Epoch 1/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 82s 267ms/step - accuracy: 0.3797 - loss: 1.4685 - val_accuracy: 0.4822 - val_loss: 1.2265
Epoch 2/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.7133 - loss: 0.7205 - val_accuracy: 0.5533 - val_loss: 1.2472
Epoch 3/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - accuracy: 0.7910 - loss: 0.5525 - val_accuracy: 0.5939 - val_loss: 1.2721
Epoch 4/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 61ms/step - accuracy: 0.8132 - loss: 0.4866 - val_accuracy: 0.6066 - val_loss: 1.3473
Epoch 5/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - accuracy: 0.8468 - loss: 0.4191 - val_accuracy: 0.6371 - val_loss: 1.3470
Epoch 6/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.8462 - loss: 0.4068 - val_accuracy: 0.6523 - val_loss: 1.3671
Epoch 7/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.8642 - loss: 0.3769 - val_accuracy: 0.6497 - val_loss: 1.3585
Epoch 8/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - accuracy: 0.8707 - loss: 0.3452 -

In [34]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.4f}")

# Import necessary libraries for evaluation metrics
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Get predictions for the test dataset
y_pred_probs = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Extract true labels from the test dataset
true_labels = np.concatenate([y for x, y in test_ds], axis=0)

# Use the class_names already available in the notebook state
# class_names = test_ds.class_names # This line caused the error

# Compute and print Confusion Matrix
cm = confusion_matrix(true_labels, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)

# Compute and print Classification Report
report = classification_report(true_labels, y_pred_classes, target_names=class_names)
print("\nClassification Report:")
print(report)

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4658 - loss: 2.6667
Test Accuracy: 0.6878
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step

Confusion Matrix:
[[ 17  47  28   8]
 [  1 102  11   1]
 [  3   7  95   0]
 [  0   7  10  57]]

Classification Report:
                  precision    recall  f1-score   support

    glioma_tumor       0.81      0.17      0.28       100
meningioma_tumor       0.63      0.89      0.73       115
        no_tumor       0.66      0.90      0.76       105
 pituitary_tumor       0.86      0.77      0.81        74

        accuracy                           0.69       394
       macro avg       0.74      0.68      0.65       394
    weighted avg       0.73      0.69      0.64       394

